# Use Twitter to find Victoria Flood Affected Suburbs

In [1]:
from twitterClient import twitterClient
import tweepy
import json

client = twitterClient()

query = '#vicfloods'
lFields = ['author_id', 'entities', 'id', 'in_reply_to_user_id', 'referenced_tweets', 'text', 'created_at',
           'public_metrics', 'lang', 'geo']

data = []

# use tweepy paginator to get more than 100 tweets, limit need to be set, here we use 10000
for i in tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100,
                          tweet_fields=lFields).flatten(limit=45000):
    data.append(i)


In [2]:
len(data)

4002

In [3]:
# save the data to json
for i in data:
    with open('data3.json', 'a+') as f:
        f.write(json.dumps(dict(i), default=str) + "\n")

In [4]:
import pandas as pd
import math
import numpy as np

df = pd.read_json('data3.json', lines=True)

df = pd.concat([df.drop(['public_metrics'],axis=1),df['public_metrics'].apply(pd.Series)],axis = 1)

df['created_at']=df['created_at'].dt.tz_convert('Australia/Melbourne')

df['Date'] = df['created_at'].dt.date
df['Time'] = df['created_at'].dt.time

df['reference_tweet1'] = ''
df['reference_tweet2'] = '' 

for i in range(0,len(df.index)):
    try:
        math.isnan(df['referenced_tweets'][i])
      
    except:
        if len(df['referenced_tweets'][i]) > 1:
            df['reference_tweet1'][i] = df['referenced_tweets'][i][0][20:39]
            df['reference_tweet2'][i] = df['referenced_tweets'][i][1][20:39]
        else:
            df['reference_tweet1'][i] = df['referenced_tweets'][i][0][20:39]

C:\Users\aaron\AppData\Local\Temp\ipykernel_5460\2708685260.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reference_tweet1'][i] = df['referenced_tweets'][i][0][20:39]
C:\Users\aaron\AppData\Local\Temp\ipykernel_5460\2708685260.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reference_tweet1'][i] = df['referenced_tweets'][i][0][20:39]
C:\Users\aaron\AppData\Local\Temp\ipykernel_5460\2708685260.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [5]:
df.head()

,entities,id,text,edit_history_tweet_ids,author_id,created_at,lang,referenced_tweets,in_reply_to_user_id,geo,retweet_count,reply_count,like_count,quote_count,Date,Time,reference_tweet1,reference_tweet2
0,"{'urls': [{'start': 105, 'end': 128, 'url': 'h...",1585593074394144768,Watch and Act - Moderate Flood Warning for Wat...,[1585593074394144768],632702595,2022-10-27 22:23:42+11:00,en,NaN,NaN,NaN,1,0,0,0,2022-10-27,22:23:42,,
1,"{'urls': [{'start': 105, 'end': 128, 'url': 'h...",1585593072531881984,Watch and Act - Moderate Flood Warning for Wat...,[1585593072531881984],1738015658,2022-10-27 22:23:42+11:00,en,NaN,NaN,NaN,1,0,0,0,2022-10-27,22:23:42,,
2,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1585588338136068097,RT @vicemergency: Minor Flood Warning - Advice...,[1585588338136068097],1499517446629527552,2022-10-27 22:04:53+11:00,en,[<ReferencedTweet id=1585586470424784896 type=...,NaN,NaN,3,0,0,0,2022-10-27,22:04:53,1585586470424784896,
3,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",1585586594202804227,RT @vicsesnews: Minor Flood Warning - Advice f...,[1585586594202804227],16881999,2022-10-27 21:57:57+11:00,en,[<ReferencedTweet id=1585586471716937728 type=...,NaN,NaN,2,0,0,0,2022-10-27,21:57:57,1585586471716937728,
4,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1585586565790978048,RT @vicemergency: Minor Flood Warning - Advice...,[1585586565790978048],19582083,2022-10-27 21:57:50+11:00,en,[<ReferencedTweet id=1585586470424784896 type=...,NaN,NaN,3,0,0,0,2022-10-27,21:57:50,1585586470424784896,


In [6]:
string_total='||'.join(df['text'].tolist())

In [7]:
dates = df['Date'].value_counts().index

In [8]:
suburbs = []
with open ("vicsuburbs.csv", 'r') as f:
    for sub in f:
        sub = sub.replace('\n','')
        suburbs.append(sub)

In [9]:
len(suburbs)

2133

In [10]:
result=[]
for sub in suburbs:
    count = string_total.lower().count(sub.lower())
    result.append([sub,count])

In [11]:
result_total = pd.DataFrame(result, columns =['sub','counts'])

In [12]:
result_total = result_total.sort_values(by='counts', ascending = False)

In [13]:
result_total.to_csv("results2.csv")

In [14]:
result_total.head(10)

,sub,counts
1290,Echuca,2034
1405,Shepparton,618
301,Melbourne,592
1398,Rochester,322
1785,Edi,322
1291,Echuca East,280
1688,Kerang,272
1404,Seymour,200
1293,Echuca Village,194
1296,Euroa,140


In [15]:
dates

Index([2022-10-22, 2022-10-23, 2022-10-21, 2022-10-24, 2022-10-25, 2022-10-26,
       2022-10-27, 2022-10-20],
      dtype='object')

In [16]:
result=[]

for date in dates:
    df_sub = df[df['Date']==date]
    string_total='||'.join(df_sub['text'].tolist())
    for sub in suburbs:
        count = string_total.lower().count(sub.lower())
        result.append([sub,date,count])

In [17]:
result_date = pd.DataFrame(result, columns =['sub','date','counts'])

In [18]:
result_wide=pd.pivot(result_date, index=['sub'], columns = 'date',values = 'counts')

In [19]:
result_wide = result_wide.sort_values(by=result_wide.columns[7], ascending = False)

In [20]:
result_wide.head(10)

date,2022-10-20,2022-10-21,2022-10-22,2022-10-23,2022-10-24,2022-10-25,2022-10-26,2022-10-27
sub,,,,,,,,
Echuca,18,276,438,548,334,278,84,58
Shepparton,8,150,66,110,94,74,58,58
Mooroopna,0,46,14,2,4,10,8,44
Warrandyte,0,0,0,0,0,0,0,26
Iona,0,10,8,4,2,10,8,26
Rochester,0,112,18,58,50,22,36,26
Kerang,4,48,62,38,48,34,14,24
Charlton,0,14,30,18,20,8,18,22
Wharparilla,0,8,12,18,14,14,12,22
